In [1]:
import glob
import sys
import os
import enum
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
plt.rcParams['figure.figsize'] = [14, 7]

In [2]:
import import_ipynb
import drivers
import prepare

importing Jupyter notebook from drivers.ipynb
importing Jupyter notebook from prepare.ipynb


In [3]:
nifty = prepare.MergedDf()

In [4]:
nifty.describe()

,Open,High,Low,Close,Shares Traded,Turnover (Rs. Cr),P/E,P/B,Div Yield
count,5225.000000,5225.000000,5225.000000,5225.000000,5.225000e+03,5225.000000,5225.000000,5225.000000,5225.000000
mean,4931.530211,4964.677742,4892.778804,4929.321809,1.515437e+08,6250.129768,19.882100,3.539041,1.419129
std,3247.573752,3256.301398,3231.571643,3243.876880,1.191109e+08,4840.834747,4.159403,0.798190,0.400195
min,853.000000,877.000000,849.950000,854.200000,1.394931e+06,40.120000,10.680000,1.920000,0.590000
25%,1667.450000,1688.250000,1644.400000,1668.750000,6.926587e+07,2620.680000,17.010000,3.020000,1.160000
50%,4877.850000,4930.250000,4833.050000,4875.050000,1.305892e+08,5462.340000,19.940000,3.470000,1.320000
75%,7588.550000,7635.550000,7532.450000,7580.200000,1.900432e+08,8149.000000,22.660000,3.800000,1.540000
max,12274.900000,12293.900000,12252.750000,12271.800000,1.414837e+09,54081.530000,29.900000,6.550000,3.180000


In [5]:
nifty.loc['1999-01-04']

Open                      896.40
High                      905.45
Low                       895.75
Close                     897.80
Shares Traded        32224833.00
Turnover (Rs. Cr)         811.39
P/E                        11.72
P/B                         2.08
Div Yield                   1.81
Name: 1999-01-04 00:00:00, dtype: float64

In [6]:
# debt = DebtCorpus()
# print(debt.Deposit(datetime(2000, 1, 1), 100))
# print(debt.Withdraw(datetime(2010, 1, 1), 100))
# debt.Get(datetime(2020, 1, 1))


# vanilla strategy with params

# monthly_sip = 100
# default_exposure = 0.5
# green_pe = 15
# red_pe = 28

# Every month, invest monthly_sip * default_exposure in index and invest monthly_sip * (1 - default_exposure) in debt.
# If nifty pe > red_pe, pull out all money from index to debt.
# if nifty pe < green_pe, pull out all money from debt to index.

In [7]:
nifty.index[-1].to_pydatetime()

datetime.datetime(2019, 12, 31, 0, 0)

In [8]:
def EvaluateStrategy(df, params):
    print('params:', json.dumps(params.__dict__, indent=2))
    push_num_installments = int(params.push_num_installments)
    pull_num_installments = int(params.pull_num_installments)
    # strategy
    curr_month = -1
    e = drivers.EquityCorpus(df)
    d = drivers.DebtCorpus()
    total_invested = 0
    num_installments = 0
    size_installment = 0;
    for ind in df.index:
        if ind.month != curr_month:
            curr_month = ind.month
            index_sip = params.monthly_sip * params.default_exposure
            debt_sip = params.monthly_sip * (1 - params.default_exposure)
            current_pe = df['P/E'][ind]
            if (current_pe < params.green_pe):
                # we are in bear market.
                if (0 == num_installments):
                    debt_funds = d.Get(ind)
                    # print('debt_funds', debt_funds, ind)
                    size_installment = debt_funds / params.push_num_installments
                to_invest = size_installment if num_installments < params.push_num_installments else d.Get(ind)
                # print('to_invest', to_invest, size_installment, d.Get(ind))
                debt_sip -= to_invest
                index_sip += to_invest
                num_installments+=1
            elif (current_pe > params.red_pe):
                # we are in bull market
                equity_funds = e.Get(ind)
                if (0 == num_installments):
                    # print('equity_funds', equity_funds, ind)
                    size_installment = equity_funds / params.pull_num_installments
                to_redeem = min(size_installment, equity_funds)
                # print('to_redeem', to_redeem, size_installment, e.Get(ind), ind)
                index_sip -= to_redeem
                debt_sip += to_redeem
                num_installments+=1
            else:
                num_installments = 0
            assert abs(index_sip + debt_sip - params.monthly_sip) < 0.01,\
                'index_sip:' + str(index_sip) + ', debt_sip: ' + str(debt_sip) + ', monthly_sip:' + str(params.monthly_sip) 
            if (index_sip > 0):
                # print('deposit in equity', index_sip, ind)
                e.Deposit(ind, index_sip)
            elif (index_sip < 0):
                # print('withdraw from equity', index_sip, ind)
                e.Withdraw(ind, - index_sip)
            if (debt_sip > 0):
                d.Deposit(ind, debt_sip)
            elif (debt_sip < 0):
                d.Withdraw(ind, - debt_sip)
            total_invested += params.monthly_sip
    start_date = df.index[0].to_pydatetime()
    end_date = df.index[-1].to_pydatetime()
    # print('start-end', start_date, end_date)
    # print('total_invested', total_invested)
    # print('e.Get()', e.Get(end_date))
    # print('d.Get()', d.Get(end_date))
    returns = (e.Get(end_date) + d.Get(end_date)) / total_invested
    print('returns', returns)
    return returns

In [9]:
# # Debug Strategy
# params = drivers.Parameters(monthly_sip=100,
#                             default_exposure=0.0,
#                             green_pe=12,
#                             red_pe=22,
#                             pull_num_installments=2.0,
#                             push_num_installments=2.0)
# EvaluateStrategy(nifty, params)

In [10]:
# Always and only equity investor
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=1,
                            green_pe=-1,
                            red_pe=100,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 1,
  "green_pe": -1,
  "red_pe": 100,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 4.4901087169736345


4.4901087169736345

In [11]:
# Always and only debt investor
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0,
                            green_pe=-1,
                            red_pe=100,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0,
  "green_pe": -1,
  "red_pe": 100,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 2.326306709798203


2.326306709798203

In [12]:
# Mixed investor, no rebalancing.
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0.5,
                            green_pe=-1,
                            red_pe=100,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0.5,
  "green_pe": -1,
  "red_pe": 100,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 3.4082077133859188


3.4082077133859188

In [13]:
# Mixed investor, with rebalancing.
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0.5,
                            green_pe=18,
                            red_pe=28,
                            pull_num_installments=12,
                            push_num_installments=12)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0.5,
  "green_pe": 18,
  "red_pe": 28,
  "pull_num_installments": 12,
  "push_num_installments": 12
}
returns 4.791885658856569


4.791885658856569

In [14]:
# A good strategy
params = drivers.Parameters(monthly_sip=100,
                            default_exposure=0.15789473684210525,
                            green_pe=16.63157894736842,
                            red_pe=24.526315789473685,
                            pull_num_installments=2.0,
                            push_num_installments=2.0)
EvaluateStrategy(nifty, params)

params: {
  "monthly_sip": 100,
  "default_exposure": 0.15789473684210525,
  "green_pe": 16.63157894736842,
  "red_pe": 24.526315789473685,
  "pull_num_installments": 2.0,
  "push_num_installments": 2.0
}
returns 11.413761644520301


11.413761644520301

In [ ]:
from scipy import optimize

def f(z, *params):
    de, gpe, rpe, pli, psi = z
    p = drivers.Parameters(monthly_sip=100,
                           default_exposure=de,
                           green_pe=gpe,
                           red_pe=rpe,
                           pull_num_installments=pli,
                           push_num_installments=psi)
    return - 1 * EvaluateStrategy(nifty, p)

rranges = ((0.0, 1.0), (12, 20), (22, 30), slice(2, 12), slice(2, 12))
resbrute = optimize.brute(f, rranges, args=None, full_output=True,
                              finish=optimize.fmin)